In [115]:
from dataclasses import dataclass
from typing import Callable
from random import randint
from random import sample
from random import random

In [116]:
@dataclass
class Cromossomo:
    dados: list[int]
    fitness: float = 0


@dataclass
class Config:
    tam_cromossomo: int
    tam_populacao: int
    maximo_iteracoes: int
    maiximo_iteracoes_sem_melhoria: int
    fitness: Callable[[Cromossomo], float]

    selecionar_pais: Callable[[list[Cromossomo]], list[tuple[Cromossomo, Cromossomo]]]

    aplicar_cruzamento: Callable[[list[tuple[Cromossomo, Cromossomo]]], list[Cromossomo]]

    aplicar_mutacao: Callable[[list[Cromossomo], float], None]
    taxa_mutacao: float

    selecionar_sobreviventes: Callable[[list[Cromossomo]], list[Cromossomo]]

In [117]:
def inicializar_populacao(tam_populacao, tam_cromossomo) -> list[Cromossomo]:
    populacao_inicial = []
    for i in range(tam_populacao):
        cromossomo = Cromossomo([randint(0, 1) for j in range(tam_cromossomo)])
        populacao_inicial.append(cromossomo)
    return populacao_inicial

In [118]:
def torneio(populacao: list[Cromossomo], tam_torneio: int = 3) -> list[tuple[Cromossomo, Cromossomo]]:
    casais = []

    for i in range(len(populacao) // 2):
        torneio1 = sample(populacao, (tam_torneio * 2))
        pai1 = sorted(torneio1, key=lambda x: x.fitness, reverse=True)[0] 
        
        populacao.remove(pai1)
        
        torneio2 = sample(populacao, tam_torneio)
        pai2 = sorted(torneio2, key=lambda x: x.fitness, reverse=True)[0] 

        casais.append((pai1, pai2))
    return casais

In [119]:
def crossover_1_corte(casais):
    filhos = []

    for par in casais:
        # Desempacota os pais
        pai1 = par[0].dados
        pai2 = par[1].dados

        # Sorteia o ponto de corte
        corte = randint(1, len(pai1) - 1)

        filho1 = pai1[:corte] + pai2[corte:] 
        filho2 = pai2[:corte] + pai1[corte:] 

        filhos.append(Cromossomo(filho1))
        filhos.append(Cromossomo(filho2))

    return filhos

In [120]:
def mutacao(cromossomos: list[Cromossomo], taxa_mutacao: float):
    for cromossomo in cromossomos:
        for i, c in enumerate(cromossomo.dados):
            if random() < taxa_mutacao:
                if c == 1:
                    cromossomo.dados[i] = 0
                else:
                    cromossomo.dados[i] = 1

In [121]:
def elitismo(populacao):
    populacao.sort(key=lambda x: x.fitness, reverse=True)
    return populacao[:len(populacao)//2]

In [122]:
def maximizar_1s(cromossomo: Cromossomo) -> int:
    """
    Função de avaliação do problema. É nessa função que o cromossomo é interpretado
    para calcular a aptidão e verificar se está atingindo o objetivo.
    """
    return sum(cromossomo.dados)

In [123]:
def algoritmo_genetico(config: Config) -> list[int]:
    # 1. t = 0
    t = 0
    melhor_individuo = None
    iteracoes_sem_melhora = 0
    
    # 2. Inicializar a população inicial P_0
    populacao = inicializar_populacao(config.tam_populacao, config.tam_cromossomo)
    populacao_antiga = populacao.copy()
    
    # 3. Enquanto critério de parada == falso
    terminou = False
    while not terminou:
    
        # a. Avaliar a população(Pt)
        print('> Avaliação da população')
        for c in populacao:
            c.fitness = config.fitness(c)
            print(c)
        
        # b. P' = Selecionar pais(Pt)
        print('> Seleção dos pais')
        casais = config.selecionar_pais(populacao)
        for c in casais:
            print(c)
            
        # c. F = Aplicar recombinação e mutação(P')
        print('> Crossover e mutação')
        F = config.aplicar_cruzamento(casais)
        config.aplicar_mutacao(F, config.taxa_mutacao)
        
        # d. Avaliar a população(F)
        print('> Avaliação da população')
        for c in F:
            c.fitness = config.fitness(c)
            print(c)
        
        # e. Pt+1 = Selecionar sobreviventes(Pt + F)
        print('> Selecionar sobreviventes')
        populacao = config.selecionar_sobreviventes(populacao + F)
        for c in populacao:
            c.fitness = config.fitness(c)
            print(c)
            
        # f. t = t + 1
        t = t + 1
        
        if populacao == populacao_antiga:
            iteracoes_sem_melhora += 1
        else:
            populacao_antiga = populacao.copy()
            iteracoes_sem_melhora = 0
        
        if t == config.maximo_iteracoes or iteracoes_sem_melhora == config.maiximo_iteracoes_sem_melhoria:
            break

In [124]:
config = Config(
    tam_cromossomo=8,

    tam_populacao=120,
    fitness=maximizar_1s,
    
    maximo_iteracoes=1000,
    maiximo_iteracoes_sem_melhoria=100,

    taxa_mutacao=0.01,
    
    selecionar_pais=torneio,

    aplicar_cruzamento=crossover_1_corte,
    aplicar_mutacao=mutacao,
    selecionar_sobreviventes=elitismo,
)

In [125]:
# Execução do algoritmo genético
solucao = algoritmo_genetico(config:q)

> Avaliação da população
Cromossomo(dados=[0, 0, 1, 1, 1, 1, 1, 0], fitness=5)
Cromossomo(dados=[1, 1, 0, 0, 1, 0, 0, 1], fitness=4)
Cromossomo(dados=[0, 1, 1, 1, 0, 1, 0, 0], fitness=4)
Cromossomo(dados=[0, 1, 1, 1, 0, 0, 1, 1], fitness=5)
Cromossomo(dados=[1, 1, 0, 0, 1, 0, 0, 1], fitness=4)
Cromossomo(dados=[1, 0, 1, 0, 0, 0, 1, 1], fitness=4)
Cromossomo(dados=[0, 1, 1, 1, 1, 1, 1, 1], fitness=7)
Cromossomo(dados=[1, 1, 0, 1, 1, 0, 0, 1], fitness=5)
Cromossomo(dados=[0, 0, 1, 1, 1, 0, 1, 0], fitness=4)
Cromossomo(dados=[1, 1, 1, 0, 1, 1, 1, 0], fitness=6)
Cromossomo(dados=[1, 1, 0, 1, 0, 1, 1, 1], fitness=6)
Cromossomo(dados=[1, 0, 0, 1, 1, 1, 0, 0], fitness=4)
Cromossomo(dados=[1, 0, 0, 0, 0, 0, 0, 0], fitness=1)
Cromossomo(dados=[0, 0, 1, 0, 1, 1, 1, 0], fitness=4)
Cromossomo(dados=[0, 1, 1, 1, 1, 0, 0, 0], fitness=4)
Cromossomo(dados=[1, 1, 1, 0, 0, 1, 1, 0], fitness=5)
Cromossomo(dados=[1, 0, 0, 1, 1, 0, 0, 0], fitness=3)
Cromossomo(dados=[1, 0, 1, 0, 1, 0, 1, 1], fitness=5)
Cro

ValueError: Sample larger than population or is negative